In [10]:
# hide
%reload_ext autoreload
%autoreload 2

In [38]:
import random

import numpy as np
import pandas as pd
import torch
import copy
from clientClass import Client
from dataFunction import *
from HE_functions import *
import tenseal as ts
from cryptotree.preprocessing import Featurizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
RANDOM_STATE = 123
from collections import OrderedDict
from base64 import b64encode, b64decode
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.special import expit
from functools import reduce
torch.random.manual_seed(11007303)
random.seed(11007303)

In [39]:
    # for (poly_mod, coeff_mod_bit_sizes) in [
    #     (8192, [40, 21, 21, 21, 21, 21, 21, 40]),
    #     (8192, [40, 20, 40]),
    #     (8192, [20, 20, 20]),
    #     (8192, [17, 17]),
    #     (4096, [40, 20, 40]),
    #     (4096, [30, 20, 30]),
    #     (4096, [20, 20, 20]),
    #     (4096, [19, 19, 19]),
    #     (4096, [18, 18, 18]),
    #     (4096, [18, 18]),
    #     (4096, [17, 17]),
    #     (2048, [20, 20]),
    #     (2048, [18, 18]),
    #     (2048, [16, 16]),
# parameters
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()
# poly_mod = 2048 
# coeff_mod_bit_sizes = [16, 16] 
# enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC #ts.ENCRYPTION_TYPE.SYMMETRIC

# context = ts.context(
#             scheme=ts.SCHEME_TYPE.CKKS,
#             poly_modulus_degree=poly_mod,
#             plain_modulus=786433,
#             coeff_mod_bit_sizes=coeff_mod_bit_sizes,
#             encryption_type=enc_type,
#         )
# context.generate_galois_keys()
# # context.serialize()

        

# Make Dataframes for every dataset

In [40]:

cleveland = "FL_HE_2/processed.cleveland.data"
switzerland = "FL_HE_2/processed.switzerland.data"
va = "FL_HE_2/processed.va.data"
hungarian = "FL_HE_2/reprocessed.hungarian.data"
cleveland_df, switzerland_df, va_df, hungarian_df = import_data(cleveland, switzerland, va, hungarian)
df_dict ={
    'Cleveland': cleveland_df
    # ,
    # 'Switzerland': switzerland_df,
    # 'VA Long Beach': va_df,
    # 'Hungary': hungarian_df        
    }



In [41]:
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [42]:
n_clients = 4
clients = []
n_features = 18
glob_model = LR(n_features)

# client_models = [copy.deepcopy(glob_model) for _ in range(n_clients)]
# client_optims = [torch.optim.SGD(model.parameters(), lr=lr) for model in client_models]
for i in list(df_dict.keys()):
    df = df_dict.get(i)
    df_dict[i] = new_df(df)
    location_data = new_df(df)
    y = location_data.HeartDisease
    location_data = location_data.drop(columns="HeartDisease")
    cat_feat = ['ChestPainType', 'RestingECG', 'ST_Slope']
    location_data = make_dummies(location_data, cat_feat)
    numeric_feature_names = ['Age', 'MaxHR', 'RestingBP',  'Cholesterol', 'Oldpeak']
    for j in numeric_feature_names:
        if location_data[j].std() != 0: 
            location_data[j] = (location_data[j] - location_data[j].mean()) / location_data[j].std()
    x = location_data
    # keys = context.generate_galois_keys()
    client_model = copy.deepcopy(glob_model)
    lr = 2
    lr_decay = 1
    clients.append(Client(i, x, y, cat_feat, client_model, lr, torch.nn.BCELoss()))



In [21]:
validation_X_set = [client.X_test for client in clients]
validation_y_set = [client.X_test for client in clients]

tensor([[-0.1592,  1.0000, -0.0960,  ...,  0.0000,  0.0000,  1.0000],
        [-0.0486,  1.0000, -1.2324,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.1727,  1.0000, -0.3801,  ...,  0.0000,  1.0000,  0.0000],
        ...,
        [ 0.2833,  1.0000, -0.0960,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.8365,  0.0000,  0.3585,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.6153,  0.0000,  1.4949,  ...,  0.0000,  0.0000,  1.0000]])

In [25]:
EPOCHS = 80

def train(model, criterion, x, y, epochs=10):
    optim = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_loss = []
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        print(f"Loss at epoch {e}: {loss.data}")
        loss.backward()
        optim.step()
    print(f"Final Loss at epoch {e}: {loss.data}")
    print(loss.item())
    epoch_loss.append(loss.item())
    return model.state_dict(), loss.item()


In [8]:
size_train_sets = []

for client in clients:
    pipe = Featurizer(cat_feat)
    y_train, y_valid, X_train_normalized, X_valid_normalized = split_prep_data(client, pipe)
    size_train_sets.append(X_train_normalized.shape[0])

In [17]:
def accuracy_loss_LR(model, x, y, criterion):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    loss = criterion(out, y)
    return correct.float().mean(), loss.item()

In [92]:
class EncryptedLR:
    
    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors,
        # so we take out the parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        
    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
        
    ################################################
    ## You can use the functions below to perform ##
    ## the evaluation with an encrypted model     ##
    ################################################
    
    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)
        
    def decrypt(self, context):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()
        



In [67]:
encrypted_weights_models = {}
encrypted_bias_models = {}
validation_X_set = []
validation_y_set = []
loss_train = []
net_best = None
best_loss = None
best_acc = None
best_epoch = None
results = []

for iter in range(EPOCHS):
    w_glob = None
    loss_locals = []
    for client in clients:
        pipe = Featurizer(cat_feat)
        y_train, y_valid, X_train_normalized, X_valid_normalized = split_prep_data(client, pipe)
        n_features = X_train_normalized.shape[1]


        
        if client.name == 'Cleveland':
            validation_X_set = X_valid_normalized
            validation_y_set = y_valid
        else:
            validation_X_set = np.concatenate((validation_X_set, X_valid_normalized), axis=0)
            validation_y_set = np.concatenate((validation_y_set, y_valid), axis=0)
        w_local, loss = train(model, optim, criterion, X_train_normalized, y_train)
        
        loss_locals.append(copy.deepcopy(loss))
        for key in w_local.keys():
            if key == 'lr.weight':
                var_list = w_local.get(key)[0].tolist()
                encrypted_values = ts.ckks_vector(ctx_eval, var_list)
                w_local[key] = encrypted_values
            else:
                var_list = w_local.get(key).tolist()
                encrypted_bias = ts.CKKSTensor(ctx_eval,var_list)
                w_local[key] = encrypted_bias
        
        if w_glob is None:
            w_glob = copy.deepcopy(w_local)
        else:
            for k in w_glob.keys():
                w_glob[k] += w_local[k]

    lr *= lr_decay

    for k in w_glob.keys():
        w_glob[k] = torch.div(w_glob[k], 4)

        for k in w_glob.keys():
        if k == 'lr.weight':
            w_glob[k] = torch.Tensor(w_glob[k].decrypt()).unsqueeze(0)
        else:
             w_glob[k] = torch.Tensor(w_glob[k].decrypt().tolist())

    net_glob.load_state_dict(w_glob)
    loss_avg = sum(loss_locals) / len(loss_locals)
    loss_train.append(loss_avg)
    if (iter + 1) % 4 == 0:
        net_glob.eval()
        acc_test, loss_test =  accuracy_loss_LR(net_glob, validation_X_set_tensor, validation_y_set_tensor.reshape(validation_y_set_tensor.shape[:2]), criterion)

        print('Round {:3d}, Average loss {:.3f}, Test loss {:.3f}, Test accuracy: {:.2f}'.format(
            iter, loss_avg, loss_test, acc_test))


        if best_acc is None or acc_test > best_acc:
            net_best = copy.deepcopy(net_glob)
            best_acc = acc_test
            best_epoch = iter

        # if (iter + 1) > args.start_saving:
        #     model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
        #     torch.save(net_glob.state_dict(), model_save_path)

        results.append(np.array([iter, loss_avg, loss_test, acc_test, best_acc]))
        final_results = np.array(results)
        final_results = pd.DataFrame(final_results, columns=['epoch', 'loss_avg', 'loss_test', 'acc_test', 'best_acc'])
        # final_results.to_csv(results_save_path, index=False)

    # if (iter + 1) % 50 == 0:
        # best_save_path = os.path.join(base_dir, 'fed/best_{}.pt'.format(iter + 1))
        # model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
        # torch.save(net_best.state_dict(), best_save_path)
        # torch.save(net_glob.state_dict(), model_save_path)

print('Best model, iter: {}, acc: {}'.format(best_epoch, best_acc))

    # validation_y_set_tensor = torch.Tensor(validation_y_set).unsqueeze(1)
    # validation_X_set_tensor = torch.tensor(validation_X_set).float()

    # plain_accuracy = accuracy_LR(net_glob, validation_X_set_tensor, validation_y_set_tensor)
    # print(f"Accuracy on plain test_set: {plain_accuracy}")
    # weights = model.lr.weight.data.tolist()[0]
    # bias = model.lr.bias.data.tolist()

    # encrypted_weights = ts.ckks_vector(ctx_eval, weights)
    # encrypted_weights_models[client.name+'_LR'] = encrypted_weights
    # encrypted_bias = ts.ckks_vector(ctx_eval, bias)
    # encrypted_bias_models[client.name+'_LR'] = encrypted_bias
    # eelr = EncryptedLR(model)       
    

Loss at epoch 1: 0.6921181678771973
Loss at epoch 2: 0.6812593340873718
Loss at epoch 3: 0.671548068523407
Loss at epoch 3: 0.671548068523407
Loss at epoch 1: 0.5236610174179077
Loss at epoch 2: 0.46238890290260315
Loss at epoch 3: 0.41394180059432983
Loss at epoch 3: 0.41394180059432983
Loss at epoch 1: 0.5577795505523682
Loss at epoch 2: 0.5494472980499268
Loss at epoch 3: 0.5426748394966125
Loss at epoch 3: 0.5426748394966125
Loss at epoch 1: 0.7966405749320984
Loss at epoch 2: 0.7686443328857422
Loss at epoch 3: 0.7452135682106018
Loss at epoch 3: 0.7452135682106018


TypeError: div(): argument 'input' (position 1) must be Tensor, not CKKSVector

In [44]:
def average_state_dict(state_dicts):
    result = copy.deepcopy(state_dicts[0])
    for key in result:
        for state_dict in state_dicts[1:]:
            result[key] += state_dict[key]
        result[key] /= len(state_dicts)
    return result

In [22]:
glob_model.state_dict()

OrderedDict([('lr.weight',
              tensor([[-0.0055, -0.0905,  0.0812, -0.1370,  0.0315, -0.0453, -0.2317, -0.1670,
                       -0.1335, -0.0807,  0.2090, -0.2217, -0.2339,  0.2296,  0.1423, -0.2275,
                        0.2180, -0.1723]])),
             ('lr.bias', tensor([0.0768]))])

In [18]:
encrypted_weights_models = {}
encrypted_bias_models = {}
validation_X_set = []
validation_y_set = []
loss_train = []
net_best = None
best_loss = None
best_acc = None
best_epoch = None
results = []

for iter in range(EPOCHS):
    w_glob = None
    loss_locals = []
    for client in clients:
        if client.name == 'Cleveland':
            y_train, y_valid, X_train_normalized, X_valid_normalized = split_prep_data(client, pipe)
            n_features = X_train_normalized.shape[1]

            # if client.name == 'Cleveland':
            #     validation_X_set = X_valid_normalized
            #     validation_y_set = y_valid
            # else:
            #     validation_X_set = np.concatenate((validation_X_set, X_valid_normalized), axis=0)
            #     validation_y_set = np.concatenate((validation_y_set, y_valid), axis=0)
            net_local = copy.deepcopy(net_glob)
            w_local, loss = train(net_local, optim, criterion, X_train_normalized, y_train)
            
            loss_locals.append(copy.deepcopy(loss))

            if w_glob is None:
                w_glob = copy.deepcopy(w_local)
            else:
                for k in w_glob.keys():
                    w_glob[k] += w_local[k]

    lr *= lr_decay

    for k in w_glob.keys():
        w_glob[k] = w_glob[k]* (1)


    
    net_glob.load_state_dict(w_glob)
    loss_avg = sum(loss_locals) / len(loss_locals)
    loss_train.append(loss_avg)
    # if (iter + 1) % 4 == 0:
    # print(w_glob)
    net_glob.eval()
    validation_y_set_tensor = y_valid
    validation_X_set_tensor = X_valid_normalized
    acc_test, loss_test =  accuracy_loss_LR(net_glob, validation_X_set_tensor, validation_y_set_tensor.reshape(validation_y_set_tensor.shape[:2]), criterion)

    print('Round {:3d}, Average loss {:.3f}, Test loss {:.3f}, Test accuracy: {:.2f}'.format(
        iter, loss_avg, loss_test, acc_test))


    if best_acc is None or acc_test > best_acc:
        net_best = copy.deepcopy(net_glob)
        best_acc = acc_test
        best_epoch = iter

    # if (iter + 1) > args.start_saving:
    #     model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
    #     torch.save(net_glob.state_dict(), model_save_path)

    results.append(np.array([iter, loss_avg, loss_test, acc_test, best_acc]))
    final_results = np.array(results)
    final_results = pd.DataFrame(final_results, columns=['epoch', 'loss_avg', 'loss_test', 'acc_test', 'best_acc'])
        # final_results.to_csv(results_save_path, index=False)

    # if (iter + 1) % 50 == 0:
        # best_save_path = os.path.join(base_dir, 'fed/best_{}.pt'.format(iter + 1))
        # model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
        # torch.save(net_best.state_dict(), best_save_path)
        # torch.save(net_glob.state_dict(), model_save_path)

print('Best model, iter: {}, acc: {}'.format(best_epoch, best_acc))



    # plain_accuracy = accuracy_LR(net_glob, validation_X_set_tensor, validation_y_set_tensor)
    # print(f"Accuracy on plain test_set: {plain_accuracy}")
    # weights = model.lr.weight.data.tolist()[0]
    # bias = model.lr.bias.data.tolist()

    # encrypted_weights = ts.ckks_vector(ctx_eval, weights)
    # encrypted_weights_models[client.name+'_LR'] = encrypted_weights
    # encrypted_bias = ts.ckks_vector(ctx_eval, bias)
    # encrypted_bias_models[client.name+'_LR'] = encrypted_bias
    # eelr = EncryptedLR(model)       
    

Loss at epoch 1: 0.4600014388561249
Loss at epoch 2: 0.45679816603660583
Loss at epoch 3: 0.4540364146232605
Loss at epoch 4: 0.45161861181259155
Loss at epoch 5: 0.4494742751121521
Loss at epoch 6: 0.4475511908531189
Loss at epoch 7: 0.4458100199699402
Loss at epoch 8: 0.4442203938961029
Loss at epoch 9: 0.4427587389945984
Loss at epoch 10: 0.4414062201976776
Final Loss at epoch 10: 0.4414062201976776
0.4414062201976776
Round   0, Average loss 0.441, Test loss 0.412, Test accuracy: 0.80
Loss at epoch 1: 0.44014784693717957
Loss at epoch 2: 0.4389713406562805
Loss at epoch 3: 0.43786653876304626
Loss at epoch 4: 0.436825156211853
Loss at epoch 5: 0.43584007024765015
Loss at epoch 6: 0.43490543961524963
Loss at epoch 7: 0.4340161681175232
Loss at epoch 8: 0.43316781520843506
Loss at epoch 9: 0.4323567748069763
Loss at epoch 10: 0.43157970905303955
Final Loss at epoch 10: 0.43157970905303955
0.43157970905303955
Round   1, Average loss 0.432, Test loss 0.403, Test accuracy: 0.80
Loss at e

In [24]:
EPOCHS = 10

In [47]:
encrypted_weights_models = {}
encrypted_bias_models = {}
validation_X_set = []
validation_y_set = []
loss_train = []
net_best = None
best_loss = None
best_acc = None
best_epoch = None
results = []

for iter in range(EPOCHS):
    loss_locals = []
    client_state_dicts = []
    for client in clients:

        # if client.name == 'Cleveland':
        #     validation_X_set = X_valid_normalized
        #     validation_y_set = y_valid
        # else:
        #     validation_X_set = np.concatenate((validation_X_set, X_valid_normalized), axis=0)
        #     validation_y_set = np.concatenate((validation_y_set, y_valid), axis=0)
        client.set_state_dict(glob_model.state_dict())
        client_state_dict, loss = client.train()
        
        loss_locals.append(copy.deepcopy(loss))

        client_state_dicts.append(client_state_dict)

    
    glob_model.load_state_dict(average_state_dict(client_state_dicts))

    
    # net_glob.load_state_dict(w_glob)
    # loss_avg = sum(loss_locals) / len(loss_locals)
    # loss_train.append(loss_avg)
    # # if (iter + 1) % 4 == 0:
    # print(w_glob)
    
    glob_model.eval()
    # validation_y_set_tensor = y_valid
    # validation_X_set_tensor = X_valid_normalized
    acc_test, loss_test =  accuracy_loss_LR(glob_model, , criterion)

    print('Round {:3d}, Average loss {:.3f}, Test loss {:.3f}, Test accuracy: {:.2f}'.format(
        iter, loss_avg, loss_test, acc_test))


    if best_acc is None or acc_test > best_acc:
        net_best = copy.deepcopy(glob_model)
        best_acc = acc_test
        best_epoch = iter

    # if (iter + 1) > args.start_saving:
    #     model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
    #     torch.save(net_glob.state_dict(), model_save_path)

    results.append(np.array([iter, loss_avg, loss_test, acc_test, best_acc]))
    final_results = np.array(results)
    final_results = pd.DataFrame(final_results, columns=['epoch', 'loss_avg', 'loss_test', 'acc_test', 'best_acc'])
        # final_results.to_csv(results_save_path, index=False)

    # if (iter + 1) % 50 == 0:
        # best_save_path = os.path.join(base_dir, 'fed/best_{}.pt'.format(iter + 1))
        # model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(iter + 1))
        # torch.save(net_best.state_dict(), best_save_path)
        # torch.save(net_glob.state_dict(), model_save_path)

print('Best model, iter: {}, acc: {}'.format(best_epoch, best_acc))



    # plain_accuracy = accuracy_LR(net_glob, validation_X_set_tensor, validation_y_set_tensor)
    # print(f"Accuracy on plain test_set: {plain_accuracy}")
    # weights = model.lr.weight.data.tolist()[0]
    # bias = model.lr.bias.data.tolist()

    # encrypted_weights = ts.ckks_vector(ctx_eval, weights)
    # encrypted_weights_models[client.name+'_LR'] = encrypted_weights
    # encrypted_bias = ts.ckks_vector(ctx_eval, bias)
    # encrypted_bias_models[client.name+'_LR'] = encrypted_bias
    # eelr = EncryptedLR(model)       
    

NameError: name 'y_valid' is not defined

In [58]:
validation_X_set.shape

torch.Size([61, 18])